In [1]:
# %load Generating_training_data_auto_using_Open_APS.py
# In[1]: Importing required libraries
get_ipython().system('pip install boto3')
get_ipython().system('pip install plotly')
import numpy as np 
import pandas as pd
import boto3
import io
import plotly.graph_objects as go
import datetime

In [2]:
# In[2]:Paginating to see all data files and get participant IDs from S3 bucket

REGION = 'us-west-2'
ACCESS_KEY_ID = '************'
SECRET_ACCESS_KEY = '*********************'
BUCKET_NAME = 'uw-nn-data'
s3c = boto3.client(
        's3', 
        region_name = REGION,
        aws_access_key_id = ACCESS_KEY_ID,
        aws_secret_access_key = SECRET_ACCESS_KEY
    )
client = boto3.client('s3')
paging_client = client.get_paginator('list_objects_v2')
result = paging_client.paginate(Bucket='uw-nn-data', Prefix='OpenAPS/',  Delimiter='/', PaginationConfig={
        'MaxItems': 1000,
    })

In [3]:
# Making a list for all participant id data that is currently available in S3
participant_ids = []
for page in result:
    if "CommonPrefixes" in page:
        for prefix in page["CommonPrefixes"]:
            participant_ids.append(prefix['Prefix'][8:-1]) 
print(participant_ids)
print(len(participant_ids)) #rechecking total number of participants

['00221634', '00309157', '00897741', '01352464', '01884126', '02033176', '03572116', '04762925', '05274556', '06593293', '06806689', '07886752', '12689381', '13029224', '13783771', '14092221', '15558575', '16975609', '17161370', '19626656', '20216809', '20396154', '20649783', '21946407', '22961398', '24448124', '24587372', '27700103', '28608066', '28756888', '32407882', '32635618', '32997134', '33470634', '34148224', '35533061', '35719805', '37764532', '37875431', '37998755', '38110191', '39986716', '40237051', '40634871', '40997757', '41263203', '41663654', '42052178', '43589707', '45025419', '45120081', '46253612', '47323535', '47750728', '52804089', '56568290', '60207627', '60844515', '61179686', '62345070', '62401782', '63047517', '64024750', '64406000', '66019205', '66773091', '66836068', '67208817', '67359234', '67539697', '68267781', '69587086', '69965708', '70454270', '70811987', '71236754', '71397255', '71618088', '73521474', '77104076', '77411181', '78420229', '79526193', '80

In [13]:
# In[3]: Creating a dictionary with participant id as the key  
         # and a list csv files (and number) for device status data as its value

devicestatus_files_across_participants = {}
num_devicestatus_files_across_participants = {}

for participant_id in participant_ids:
    num_devicestatus_files_for_this_participant = 0
    devicestatus_files_for_this_participant = []

    result = paging_client.paginate(Bucket='uw-nn-data', Prefix='OpenAPS/' + participant_id, PaginationConfig={
        'MaxItems': 1000,
    })

    for page in result:
        if "Contents" in page:
            for key in page["Contents"]:
                if 'devicestatus' in key['Key']:
                    num_devicestatus_files_for_this_participant += 1
                    devicestatus_files_for_this_participant.append(key['Key'])
    num_devicestatus_files_across_participants[participant_id] = num_devicestatus_files_for_this_participant
# Filter for only those participants which have enough devicestatus data for analysing now
    if num_devicestatus_files_for_this_participant ==8 : #based on how much devicestatus data we need, please change here
        devicestatus_files_across_participants[participant_id] = devicestatus_files_for_this_participant
        print(participant_id)
print(len(devicestatus_files_across_participants))


02033176
04762925
14092221
16975609
84081904
99712241
6


In [14]:
#how many maximam device status files we have 
max_num_devicestatus_files_across_participants = 0
for participant_id in num_devicestatus_files_across_participants:
    if num_devicestatus_files_across_participants[participant_id] > max_num_devicestatus_files_across_participants:
        max_num_devicestatus_files_across_participants = num_devicestatus_files_across_participants[participant_id]
print(max_num_devicestatus_files_across_participants)

20


In [15]:
#how many minimum device status files we have 
min_num_devicestatus_files_across_participants = 1
for participant_id in num_devicestatus_files_across_participants:
    if num_devicestatus_files_across_participants[participant_id] < min_num_devicestatus_files_across_participants:
        min_num_devicestatus_files_across_participants = num_devicestatus_files_across_participants[participant_id]
        if num_devicestatus_files_across_participants[participant_id] == 0:
            print(participant_id)
print(min_num_devicestatus_files_across_participants)
#just analysing what are maximam and min number of device status files from our S3 bucket data

32635618
0


In [16]:
# In[4]:Creating a dict with participant IDs key 
       #and list of with only those device status files where eventualBG, bg, rate columns and timestamps appear as its value
    
devicestatus_files_to_be_merged_for_this_participant_across_participants = {}

for participant in devicestatus_files_across_participants:
    devicestatus_files_to_be_merged_for_this_participant = []
    
    # Iterate through all device status files for this participant and append
    # eligible files with bg, eventual bg and rate to devicestatus_files_to_be_merged_for_this_participant
    for file in devicestatus_files_across_participants[participant]:
        obj = s3c.get_object(Bucket= 'uw-nn-data', Key = file)
        lines = obj['Body'].read().decode('utf-8').splitlines(True)
        column_names = lines[0].split(",")
        if ('openaps/enacted/bg' in column_names) and ('openaps/enacted/eventualBG' in column_names) and ('openaps/enacted/rate' in column_names) and ('openaps/enacted/timestamp' in column_names):
            devicestatus_files_to_be_merged_for_this_participant.append(file) #when we need mor columns, just add their names here

    # Now assign all appended eligible files list as value to respective participant id as key 
    # in dict devicestatus_files_to_be_merged_for_this_participant_across_participants, 
    #if there are no such device status files, we don't those participants at this point
    if len(devicestatus_files_to_be_merged_for_this_participant) >0 :
        devicestatus_files_to_be_merged_for_this_participant_across_participants[participant] = devicestatus_files_to_be_merged_for_this_participant 

In [17]:
# In[5]:Creating dict with participant ID as key and list of training data files as its key 
        #by reading csv device statusdata files in S3 to create training data files each with 4 headers in original data 
        #'openaps/enacted/timestamp','openaps/enacted/bg','openaps/enacted/eventualBG','openaps/enacted/rate' and remove NAN values
devicestatus_training_files_to_be_merged_for_each_participant = {}
for participant in devicestatus_files_to_be_merged_for_this_participant_across_participants:
    devicestatus_training_files_to_be_merged_for_this_participant=[]
    for file in devicestatus_files_to_be_merged_for_this_participant_across_participants[participant]:
        obj = s3c.get_object(Bucket= 'uw-nn-data', Key = file)
        reading_device_status_eligible_file = pd.read_csv(io.BytesIO(obj['Body'].read()), encoding='utf8')
        reading_device_status_eligible_file_train = reading_device_status_eligible_file.loc[:,['openaps/enacted/timestamp','openaps/enacted/bg','openaps/enacted/eventualBG','openaps/enacted/rate']].drop_duplicates()
        reading_device_status_eligible_file_train.dropna(inplace=True)
        devicestatus_training_files_to_be_merged_for_this_participant.append(reading_device_status_eligible_file_train)
    devicestatus_training_files_to_be_merged_for_each_participant[participant] = devicestatus_training_files_to_be_merged_for_this_participant

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (228,229,230,232,233,240,243,246,249,250,254,257,266,441,444) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (463) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (13,20,23,41,331,450) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (308) have mixed types.Specify dtype option on import or set low_memory=False.
 

In [18]:
# In[6]:concatenating all eligible files listed as value in devicestatus_training_files_to_be_merged_for_each_participant[participant]  
devicestatus_training_files_combined_for_each_participant = {}
for participant in devicestatus_training_files_to_be_merged_for_each_participant:
   #defining empty dataframe to concatenate all device status files in to one trainin dataset for each participant
    devicestatus_training_files_combined_for_this_participant = pd.DataFrame()   
    for file in devicestatus_training_files_to_be_merged_for_each_participant[participant]:
        if devicestatus_training_files_combined_for_this_participant.empty:
            devicestatus_training_files_combined_for_this_participant = file
        else:
            devicestatus_training_files_combined_for_this_participant = pd.concat([devicestatus_training_files_combined_for_this_participant, file], ignore_index=True)
    devicestatus_training_files_combined_for_each_participant[participant]=devicestatus_training_files_combined_for_this_participant


In [19]:
print (len(devicestatus_training_files_combined_for_each_participant))

6


In [21]:
# In[6]:Creating a dict with participant ID as key and one single concantenated list 
#of all device status training files with updated time column data type 
devicestatus_precombined_training_files_convertedto_timedelta = {}
for participant in devicestatus_training_files_combined_for_each_participant:    
    # All files for each participant need to convert time column openaps/enacted/timestamp requires 
    #to be converted to different (timedelta) data type in 
    #training data  which is value in devicestatus_training_files_to_be_merged_for_each_participant
    file = devicestatus_training_files_combined_for_each_participant[participant]
    #time format in all device status files is not uniform, so far, I have found two formats %Y-%m-%dT%H:%M:%S.%fZ and %Y-%m-%dT%H:%M:%fZ
    file['openaps/enacted/timestamp'] = file['openaps/enacted/timestamp'].apply(lambda x: pd.to_datetime(x))
    file = file.sort_values(by=['openaps/enacted/timestamp'])   
    print(file)
#     file['openaps/enacted/timedelta'] = file['openaps/enacted/timestamp'].diff().dt.total_seconds()   
#     file['openaps/enacted/timedelta'].iloc[0] = 0
#         #print(file['openaps/enacted/timedelta'])
#     file.pop('openaps/enacted/timestamp')           
    devicestatus_precombined_training_files_convertedto_timedelta[participant] = file

           openaps/enacted/timestamp  openaps/enacted/bg  \
215 2019-06-19 12:32:51.992000+00:00               195.0   
225        2019-06-19 13:45:01+00:00               144.0   
224        2019-06-19 13:50:00+00:00               139.0   
223        2019-06-19 13:51:03+00:00               139.0   
222        2019-06-19 14:07:10+00:00               121.0   
..                               ...                 ...   
4          2019-06-22 12:11:36+00:00               121.0   
3          2019-06-22 12:25:53+00:00               122.0   
2          2019-06-22 14:09:49+00:00               177.0   
1   2019-06-22 14:43:29.942000+00:00               188.0   
0   2019-06-22 14:54:13.601000+00:00               162.0   

     openaps/enacted/eventualBG  openaps/enacted/rate  
215                        39.0                   0.0  
225                       183.0                   0.8  
224                       176.0                   0.0  
223                       177.0                   0.8  

In [22]:
# In[7]: Writing all training data as csv files in local directory 
for participant in devicestatus_precombined_training_files_convertedto_timedelta:
     devicestatus_precombined_training_files_convertedto_timedelta[participant].to_csv('data/DATA_timestamps/' + participant + '.csv', index=False)